In [1]:
import json
import requests
from tqdm import tqdm
from xml.etree import ElementTree
from clarin_morphoDita import get_word_base_and_tag

text = "A mogę, bo moim zdaniem jest do niczego. I to delikatnie mówiąc... Rzecz gustu :) Ja sobie z przyjemnością obejrzałam wczoraj \"Live and Let Die\". Nareszcie bez Connery'ego!!! Pasuje. Najbardziej ze wszystkich. Może trochę za ładny, ale grą pasuje. IMHO. A Connery to nie święta krowa ;)"

In [2]:
with open('train-raw.txt') as f:
    lines = list(filter(lambda x: x != "", [line.rstrip() for line in f]))
tree = ElementTree.parse('train-gold.xml')
root = tree.getroot()

In [3]:
chunks = tree.find("chunkList").findall("chunk")

matched = 0
miss_matched = 0
CTAG_POSSITION = 2
BASE_POSSITION = 1
TEXT_POSSITION = 0

chunk_id = 0
for line in tqdm(chunks):
    curr_line = get_word_base_and_tag(lines[chunk_id])
#     print(len(curr_line), curr_line)
    word_id = 0
    
    for sentence in line.findall("chunk"):
        
        for word in sentence.findall("tok"):
            if word_id >= len(curr_line):
                miss_matched += 1
                break
            
            curr_word = curr_line[word_id]
            orth = word.find("orth").text
            base = word.find("lex").find("base").text
            ctag = word.find("lex").find("ctag").text

#             print(f"{orth} {base} {ctag}")
#             print(curr_word)
            if ctag == curr_word[CTAG_POSSITION]:
                matched += 1
            else:
#                 print(orth, ctag)
#                 print(curr_word[TEXT_POSSITION], curr_word[CTAG_POSSITION])
                miss_matched += 1
            word_id += 1
            
    chunk_id += 1

100%|██████████| 18484/18484 [3:05:08<00:00,  1.66it/s]  


In [4]:
print(matched, miss_matched)

1073159 141838


In [5]:
matched/(matched+miss_matched)

0.8832606171044044